In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Hello World: Ang Iyong Unang Quantum Circuit

Bumuo ng [Bell state](https://en.wikipedia.org/wiki/Bell_state) (dalawang magkakaugnay na qubit) at patakbuhin ito sa tatlong paraan:

1. **Ideal simulation** — perpektong mga resulta, walang kinakailangang account
2. **Noisy simulation** — sumusubok ng tunay na device, walang kinakailangang account
3. **Tunay na quantum hardware** — nangangailangan ng [IBM Quantum account](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account)

## Bumuo ng circuit

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

qc.draw(output="mpl")

## Opsyon 1: Ideal simulation (walang kinakailangang account)
Gumagamit ng `StatevectorSampler` — isang lokal na simulator na may perpekto at walang-ingay na mga resulta.

In [ ]:
from qiskit.primitives import StatevectorSampler

result = StatevectorSampler().run([qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

## Opsyon 2: Noisy simulation (walang kinakailangang account)
Gumagamit ng `FakeManilaV2` — isang lokal na simulator na tumutulad sa tunay na IBM quantum device, kasama ang mga katangian ng ingay nito. Dapat munang i-transpile (i-adjust) ang circuit sa gate set at qubit connectivity ng device.

In [ ]:
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = FakeManilaV2()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Opsyon 3: Tunay na quantum hardware
Nangangailangan ng IBM Quantum account. Tingnan ang [Setting up your IBM Quantum account](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account) para sa mga detalye.

Kung hindi mo pa nase-save ang iyong mga credentials sa Binder session na ito, patakbuhin muna ito:

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    token="<your-api-key>",
    instance="<your-crn>",
    overwrite=True
)

**Paalala:** Ang mga job sa tunay na hardware ay maaaring tumagal ng ilang oras depende sa queue times. Kung tumatakbo pa rin ang cell, maaari ninyong tingnan ang inyong job status at makita ang mga resulta sa [quantum.cloud.ibm.com/workloads](https://quantum.cloud.ibm.com/workloads?user=me).

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(f"Running on {backend.name}")

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Ano ang susunod?
- **[Tutorials](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=docs/tutorials)** — sunud-sunod na mga gabay sa mga algorithm, error mitigation, transpilation, at iba pa
- **[Courses](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=learning/courses)** — nakaayos na mga landas ng pag-aaral mula sa mga pangunahing quantum hanggang sa utility-scale computing
- **[Local testing mode](https://janlahmann.github.io/Qiskit-documentation/#no-token-use-local-testing-mode)** — magpatakbo ng karamihan sa mga notebook nang walang IBM Quantum account